# Laboratorio de regresión - 5

|                |   |
:----------------|---|
| **Nombre**     | Eddie Aguilar |
| **Fecha**      | 17/02/2025  |
| **Expediente** | 739352  |

## Validación

Hemos estado usando `train_test_split` en nuestros modelos anteriores.

¿Por qué?

Ya que es conveniente dividir un modelo en subset de entrenamiento y un subset de validación, esto para poder saber si el modelo planteado se ajusta bien. 

Si la muestra es un subset de la población y queremos generalizar sobre la población, ¿no sería mejor utilizar todos los datos al entrenar un modelo?

Ya que en ese caso no tendríamos datos con los cuales validar si nuestro modelo se ajusta bien o no

El propósito de volver a muestrear dentro de nuestro dataset es tener una idea de qué tan buena podría ser la generalización de nuestro modelo. Imagina un dataset ya separado en dos mitades. Utilizas la primera mitad para entrenar el modelo y pruebas en la segunda mitad; la segunda mitad eran datos invisibles para el modelo al momento de entrenar. Esto nos lleva a tres escenario típicos:

1. Si el modelo hace buenas predicciones en la segunda mitad, significa que la primera mitad era "suficiente" para generalizar.
2. Si el modelo no hace buenas predicciones en la segunda mitad, pero sí en la primera mitad, podría ser que había información importante en la segunda mitad que debió haber sido tomada en cuenta al entrenar, o un problema de overfitting.
3. Si el modelo no hace buenas predicciones en la segunda mitad, y tampoco en la primera mitad, se tendrían que revisar los factores y/o el modelo seleccionado.

El caso ideal sería el 1, pero por estadística los errores y varianzas tienen como entrada el número de muestas, por lo que tenemos menos seguridad de nuestros resutados al usar menos muestras. Si vemos que el modelo generaliza bien podemos unir de nuevo el dataset y entrenar sobre el dataset completo.

En el caso 2 está el problema de que no podemos saber qué información es necesaria para el entrenamiento apropiado del modelo; esto nos lleva a pensar que debemos usar el dataset completo para entrenar, pero esto nos lleva al mismo problema de no saber si el modelo puede generalizar.

El problema sólo incrementa si se tienen hiperparámetros en el modelo (e.g. $\lambda$ en regularización).

## Leave-One-Out Cross Validation

Este método de validación es una colección de $n$ `train-test-split`. Teniendo un dataset de $n$ muestras, la lógica es:
1. Saca una muestra del dataset.
2. Entrena tu modelo con las $n-1$ muestras.
3. Evalúa tu modelo en la muestra que quedó fuera con el métrico que más se ajuste a la aplicación.
4. Regresa la muestra al dataset.
5. Repite 1-4 con muestras diferentes hasta haber hecho el procedimiento $n$ veces para $n$ muestras.
6. Calcula la media y desviación estándar de los métricos guardados.

Con los resultados del proceso de validación podemos saber qué tan bueno podría ser el modelo seleccionado con los datos (con/sin transformaciones).

### Ejercicio 1

Utiliza el dataset `Motor Trend Car Road Tests`. Elimina la columna `model` y entrena 32 modelos diferentes utilizando Leave-One-Out Cross Validation con target `mpg`. Utiliza MSE como métrico.

In [48]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error

data = pd.read_excel(r"C:\Users\AgJo413\Documents\GitHub\Lab_std\labstds\Data\Motor Trend Car Road Tests.xlsx")

In [49]:
data.head()

,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [50]:
data = data.drop(columns=["model"])

In [51]:
def modeling(trainx, trainy, testx, testy):
    model = LinearRegression()
    model.fit(trainx, trainy)
    ypred = model.predict(testx)
    return ypred

def MSE(ypred, testy):
    return mean_squared_error(testy, ypred)

In [52]:
mses = []
for i in range(len(data)):
    train = data.iloc[data.index != i]
    test = data.iloc[[i], :]
    trainx = train.loc[:, train.columns != "mpg"]
    trainy = train.loc[:, train.columns == "mpg"]
    testx = test.loc[:, test.columns != "mpg"]
    testy = test.loc[:, test.columns == "mpg"]
    ypred = modeling(trainx, trainy, testx, testy)
    mses.append(MSE(ypred, testy))

print("MSE has an average of: {}".format(np.mean(mses)))
print("MSE has a standard deviation of: {}".format(np.std(mses)))
    

MSE has an average of: 12.181558006901943
MSE has a standard deviation of: 17.067399871888504


Interpreta.

In [53]:
X = data.drop(columns=["mpg"])
y = data["mpg"]

model = LinearRegression()
model.fit(X, y)
ypred = model.predict(X)

In [54]:
MSE(ypred, y)

4.609200938020331

El promedio de MSE dentro de esta cross-validation nos ayuda para poder comprar nuestro modelo con esta media, de esta forma, en teoría nuestro modelo con todos los datos debería tener mejor calificación/valor que este promedio. 

Si la desviación estándar es pqueña significa que los folders no cambia mucho de uno a otro. Si es alta significa que hay folds en los que estás dejando información importante o que tiene mucha importancia.

## K-Folds Cross-Validation

El dataset `Motor Trend Car Road Tests` sólo tiene 32 muestras, y utilizar un modelo sencillo de regresión múltiple hace que usar LOOCV sea muy rápido. El dataset `California Housing` tiene $20640$ muestras para $9$ columnas, entonces realizar un ajuste sobre una transformación o sobre el modelo y luego calcular el impacto esperado podría tomar más tiempo.

La solución propuesta es dividir el dataset en *k* folds (partes iguales), ajustar en *k-1* folds y probar en el restante.

### Ejercicio 2
Utiliza el dataset `California Housing` y haz K-folds Cross Validation con 10 folds. Utiliza el MSE como métrico.

In [ ]:


from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print("Dataset Shape:", housing.data.shape, housing.target.shape)
print("Dataset Features:", housing.feature_names)
print("Dataset Target:", housing.target_names)
X = housing.data
y = housing.target

Dataset Shape: (20640, 8) (20640,)
Dataset Features: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
Dataset Target: ['MedHouseVal']
(20640, 8)


In [ ]:

data = pd.DataFrame(X, columns=housing.feature_names)
data[housing.target_names[0]] = y
data.head()

(20640,)


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [57]:
data_shuffled = data.sample(frac=1)
data_shuffled.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
12090,2.9542,15.0,6.074581,1.143075,2317.0,3.526636,33.85,-117.30,1.27900
8854,15.0001,52.0,8.710145,1.028986,408.0,2.956522,34.07,-118.41,5.00001
16294,1.5539,44.0,5.066667,1.087356,1277.0,2.935632,37.96,-121.23,0.59200
1950,2.2269,17.0,5.020436,1.055858,1714.0,2.335150,38.74,-120.83,1.28300
9712,3.6875,19.0,4.979277,1.052072,4259.0,2.263018,36.66,-121.67,1.89700


In [ ]:
mses = []
for i in range(0, len(data_shuffled), 10):
    test = data_shuffled.iloc[i:i+10, :]
    train = data_shuffled.drop(test.index)
    trainx = train.loc[:, train.columns != "MedHouseVal"]
    trainy = train.loc[:, train.columns == "MedHouseVal"]
    testx = test.loc[:, test.columns != "MedHouseVal"]
    testy = test.loc[:, test.columns == "MedHouseVal"]
    ypred = modeling(trainx, trainy, testx, testy)
    mses.append(MSE(ypred, testy))

print("MSE has an average of: {}".format(np.mean(mses)))
print("MSE has a standard deviation of: {}".format(np.std(mses)))
    

MSE has an average of: 0.528237096685978
MSE has a standard deviation of: 0.43472862510176086


In [64]:
X = data.drop(columns=["MedHouseVal"])
y = data["MedHouseVal"]

model = LinearRegression()
model.fit(X, y)
ypred = model.predict(X)

In [65]:
MSE(ypred, y)

0.5243209861846072

Interpreta.

El promedio de MSE dentro de esta cross-validation nos ayuda para poder comprar nuestro modelo con esta media, de esta forma, en teoría nuestro modelo con todos los datos debería tener mejor calificación/valor que este promedio. 

Si la desviación estándar es pqueña significa que los folders no cambia mucho de uno a otro. Si es alta significa que hay folds en los que estás dejando información importante o que tiene mucha importancia.

En este caso, nuestro modelo está muy por debajo del promedio, entonces puede ser buena idea replantear ese modelo. Aunque también la significacnia o relevancia de esta delta es directmanete proporcional a la desviación estándar, una pequeña desviación estandar podría justificar un delta muy pequeño. Un siguiente paso podría ser hacer una prueba de hipotesis donde planteemos si esta diferencia realmente es dependiente al cambio de columnas. 

## Referencia

James, G., Witten, D., Hastie, T., Tibshirani, R.,, Taylor, J. (2023). An Introduction to Statistical Learning with Applications in Python. Cham: Springer. ISBN: 978-3-031-38746-3